In [ ]:
import pandas as pd
import os
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter


In [ ]:
import os
os.getcwd()

In [ ]:
import pandas as pd
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image

class ImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=True, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)[0:7000]
        self.img_dir = img_dir
        self.imgsize =(256 ,256)
        self.transform = transforms.Compose([transforms.Resize(self.imgsize),transforms.ToTensor()])
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])+'.jpg'
        image = Image.open(img_path).convert('RGB')
        label = self.img_labels.iloc[idx, -1]/100
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    
class test_ImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=True, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)[8000:]
        self.img_dir = img_dir
        self.imgsize =(256 ,256)
        self.transform = transforms.Compose([transforms.Resize(self.imgsize),transforms.ToTensor()])
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])+'.jpg'
        image = Image.open(img_path).convert('RGB')
        label = self.img_labels.iloc[idx, -1]/100
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
from torch.utils.data import DataLoader
train_data = ImageDataset(annotations_file='../input/petfinder-pawpularity-score/train.csv',img_dir="../input/petfinder-pawpularity-score/train/")
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)

test_data = test_ImageDataset(annotations_file='../input/petfinder-pawpularity-score/train.csv',img_dir="../input/petfinder-pawpularity-score/train/")
test_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)

In [ ]:
from torch import nn
import torchvision.models as models

# model = models.resnet18(pretrained=False)
# model.fc = nn.Sequential(
#                nn.Linear(512, 1000),
#                nn.ReLU(inplace=True),
#                nn.Linear(1000, 125),
#                 nn.Linear(125, 1))
model = models.resnet50(pretrained=False)
model.fcc = nn.Sequential(
               nn.ReLU(inplace=True),
               nn.Linear(1000, 125),
                nn.Linear(125, 1))

In [ ]:
model

In [ ]:
from torchvision import models
import torch.nn as nn
from torch.optim import Adam


optimizer = Adam(model.parameters(), lr=0.0003)

In [ ]:
import wandb
wandb.login()
wandb.init(project='PetFinder_my_Pawpularity resnet50_Pretrain')
config = wandb.config
config.learn_rate = 0.0003
config.Opt = 'Adam'
config.lossfunction = 'RMES'

In [ ]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#model.load_state_dict(torch.load('../input/pet-pt/pet_weight.pt'))
model.to(device)

In [ ]:
#training model

minloss = 100
train_losses = []
# empty list to store validation losses
val_losses = []
epochs = 1000
wandb.watch(model)
for epoch in range(epochs+1):
    model.train()
    tr_loss = 0
    total =0
    total_train = 0
    # getting the training set
    x_train, y_train = next(iter(train_dataloader))
    # getting the validation set
    x_val, y_val = next(iter(test_dataloader))

    # clearing the Gradients of the model parameters
    optimizer.zero_grad()
    
    # prediction for training and validation set
    output_train = model(x_train.to(device))
    output_val = model(x_val.to(device))
    
    
    # computing the training and validation loss
    criterion = nn.MSELoss()
    output_train = output_train.to(torch.float32)
    y_train = y_train.to(device).to(torch.float32)
    loss_train = torch.sqrt(criterion(output_train, y_train))
    loss_val = torch.sqrt(criterion(output_val, y_val.to(device)))

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    train_loss = loss_train.item()
    wandb.log({'train_loss':loss_train,'val_loss':loss_val})
    
    if epoch%10 == 0:
        # printing the validation loss
        print('Epoch : ',epoch)
        print('train_loss',loss_train.item(),'val_loss :', loss_val.item())
    if minloss > loss_val.item():
        minloss = loss_val.item()
        torch.save(model.state_dict(), 'pet_weight.pt')
        

In [ ]:
# test loader
class test_ImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=True):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.imgsize =(256 ,256)
        self.transform = transforms.Compose([transforms.Resize(self.imgsize),transforms.ToTensor()])
        

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])+'.jpg'
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image

In [ ]:
test_data = test_ImageDataset(annotations_file="../input/petfinder-pawpularity-score/test.csv",img_dir="../input/petfinder-pawpularity-score/test/")
test_dataloader = DataLoader(test_data, shuffle=False)

In [ ]:
# test 
pred = []

with torch.no_grad():
    model.eval()
    for i in test_dataloader:
        y = model(i.to(device)).cpu().numpy()[0][0]
        pred.append(y)
        
        print(y)
d = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
d['Pawpularity'] = pred

In [ ]:
submission_df = d[['Id','Pawpularity']]
submission_df.to_csv("./submission.csv", index=False)
submission_df.head()